In [8]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("final_performance_v4.csv", parse_dates=['방송일시','date', 'real_date'])
data

,Unnamed: 0,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,...,g_3,s_0,s_1,w_0,w_1,w_2,w_3,w_4,w_5,cpi
0,0,2019-01-01 06:00:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,2099000.0,2019-01-01,...,0,0,0,0,0,0,0,0,0,104.03
1,1,2019-01-01 06:00:00,20.0,296,898,테이트 여성 셀린니트3종,6,39900,4371000.0,2019-01-01,...,0,0,0,0,0,0,0,0,0,104.03
2,2,2019-01-01 06:20:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,3262000.0,2019-01-01,...,0,1,0,0,0,0,0,0,0,104.03
3,3,2019-01-01 06:20:00,20.0,296,898,테이트 여성 셀린니트3종,6,39900,6955000.0,2019-01-01,...,0,1,0,0,0,0,0,0,0,104.03
4,4,2019-01-01 06:40:00,20.0,296,891,테이트 남성 셀린니트3종,6,39900,6672000.0,2019-01-01,...,0,0,1,0,0,0,0,0,0,104.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,35374,2019-12-31 23:40:00,20.0,374,1130,일시불쿠첸압력밥솥 6인용,9,148000,10157000.0,2019-12-31,...,1,1,0,0,0,0,0,0,0,105.51
35375,35375,2020-01-01 00:00:00,20.0,374,1124,무이자쿠첸압력밥솥 10인용,9,178000,50929000.0,2019-12-31,...,1,0,1,0,0,0,0,0,0,106.26
35376,35376,2020-01-01 00:00:00,20.0,374,1129,일시불쿠첸압력밥솥 10인용,9,168000,104392000.0,2019-12-31,...,1,0,1,0,0,0,0,0,0,106.26
35377,35377,2020-01-01 00:00:00,20.0,374,1125,무이자쿠첸압력밥솥 6인용,9,158000,13765000.0,2019-12-31,...,1,0,1,0,0,0,0,0,0,106.26


In [3]:
import sklearn
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import gc

In [4]:
use_var = ['마더코드', '상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage', 'prime', 'IsHoliday', '노출(분)', '판매단가', '취급액', '지속휴일수', 'TEMP', 'HUM', 'cpi']
Data = data[use_var]

In [5]:
X = Data.drop(['취급액'], axis = 1)
Y = Data[['취급액']]
print("data shape : ", X.shape)
print("label shape : " , Y.shape)

data shape :  (35379, 15)
label shape :  (35379, 1)


In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.1, random_state = 123457)
print("x_train shape : ", x_train.shape)
print("y_train shape : ", y_train.shape)
print("x_valid shape : ", x_valid.shape)
print("y_valid shape : ", y_valid.shape)

x_train shape :  (31841, 15)
y_train shape :  (31841, 1)
x_valid shape :  (3538, 15)
y_valid shape :  (3538, 1)


## Make train data

In [18]:
make_train_cols = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage', 'IsHoliday', '노출(분)', '판매단가', '지속휴일수', 'TEMP', 'HUM', 'cpi']
to_cat = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage']
train_goods_set = set(x_train['상품코드'])
for col in to_cat:
    x_train[col] = x_train[col].astype('category') 
    
print("x_train shape : ", x_train.shape)

x_train shape :  (31841, 15)


## Make valid data

In [19]:
pd.set_option('mode.chained_assignment',  None)
def goods_code_encoding(value,code_set, price, mother, cat):
    if value in code_set:
        return value
    else:
        if mother in x_train['마더코드']:
            frame = x_train[x_train['마더코드']==mother]
            frame['diff'] =  abs(frame['판매단가'] - price)
            goods_code = frame[frame['diff']==min(frame['diff'])].iloc[0,1]
            return goods_code
        else:
            frame = x_train[x_train['상품군']==cat]
            frame['diff'] =  abs(frame['판매단가'] - price)
            goods_code = frame[frame['diff']==min(frame['diff'])].iloc[0,1]
            return goods_code     

In [20]:
make_valid_cols = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage', 'IsHoliday', '노출(분)', '판매단가', '지속휴일수', 'TEMP', 'HUM', 'cpi']
to_cat = ['상품코드', '상품군', 'week', 'hour', 'weekofyear', 'stage']

row = x_valid.iloc[0]
x_valid.iloc[0] = goods_code_encoding(row['상품코드'],train_goods_set, row['판매단가'], row['마더코드'], row['상품군'])
for i in range(1, len(x_valid)):
    row = x_valid.iloc[i]
    x_valid.iloc[i] = goods_code_encoding(row['상품코드'], train_goods_set, row['판매단가'], row['마더코드'], row['상품군'])

for col in to_cat:
    x_valid[col] = x_valid[col].astype('category') 
    
print("x_valid shape : ", x_valid.shape)

x_valid shape :  (3538, 15)


## lightgbm 데이터셋으로 변환

In [21]:
import lightgbm as lgb

train_ds = lgb.Dataset(x_train, label = y_train) 
valid_ds = lgb.Dataset(x_valid, label = y_valid)

## 파라미터 세팅

In [65]:
params = {'learning_rate': 0.1, 
          'max_depth': -1, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mape', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':123457}

## 모델 훈련

In [66]:
model = lgb.train(params, train_ds, 100000, valid_ds, verbose_eval=100, early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003891 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2897
[LightGBM] [Info] Number of data points in the train set: 31841, number of used features: 15
[LightGBM] [Info] Start training from score 23136023.429917
Training until validation scores don't improve for 100 rounds
[100]	valid_0's mape: 1.13379
[200]	valid_0's mape: 0.97807
[300]	valid_0's mape: 0.939808
[400]	valid_0's mape: 0.846244
[500]	valid_0's mape: 0.844504
[600]	valid_0's mape: 0.749195
[700]	valid_0's mape: 0.703048
[800]	valid_0's mape: 0.67618
[900]	valid_0's mape: 0.670114
[1000]	valid_0's mape: 0.665066
Early stopping, best iteration is:
[970]	valid_0's mape: 0.664944


In [67]:
predict_train = model.predict(x_train).reshape(-1, 1)
predict_test = model.predict(x_valid).reshape(-1, 1)

In [68]:
import numpy as np

def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [69]:
train_mape = MAPE(y_train, predict_train)
test_mape = MAPE(y_valid, predict_test)

print('train data MAPE: ', train_mape)
print('test data MAPE: ', test_mape)

train data MAPE:  18.250669432923537
test data MAPE:  66.49439393798862
